In [1]:
import rolch
import numpy as np
from sklearn.datasets import load_diabetes, make_regression
import matplotlib.pyplot as plt

np.set_printoptions(precision=3, suppress=True)
print(rolch.__version__)

0.1.8


# Batch Estimation

In [2]:
## Diabetes data set
## Add intercept (will not be regularized)

X, y = load_diabetes(return_X_y=True)
X = np.hstack((np.ones((X.shape[0], 1)), X))

## OLS

In [3]:
online_gamlss_ols = rolch.OnlineGamlss(
    distribution=rolch.DistributionNormal(), method="ols", rss_tol_inner=np.inf
)

online_gamlss_ols.fit(y, X, X)

print("OLS Coefficients \n")
print(np.vstack(online_gamlss_ols.betas).T)

OLS Coefficients 

[[151.647   3.94 ]
 [  0.576  -0.066]
 [-11.585  -0.114]
 [ 22.359   0.046]
 [ 16.478   0.108]
 [-25.09   -0.41 ]
 [ 12.188   0.429]
 [ -0.676  -0.073]
 [  6.9    -0.172]
 [ 30.304   0.206]
 [  2.88    0.057]]


## LASSO

In [4]:
dist = rolch.DistributionT()

online_gamlss_lasso = rolch.OnlineGamlss(
    distribution=dist,
    method="lasso",
    estimation_kwargs={
        "ic": {i: "bic" for i in range(dist.n_params)},  # Change the IC if you like
        "lambda_eps": {
            i: 1e-4 for i in range(dist.n_params)
        },  # Change the design of the lambda grid
    },
    rss_tol_inner=np.inf,
)
online_gamlss_lasso.fit(y, X, X, X)

print("LASSO Coefficients \n")
print(np.vstack(online_gamlss_lasso.betas).T)

LASSO Coefficients 

[[151.968   3.973  25.   ]
 [ -0.     -0.     -0.   ]
 [ -9.826  -0.      0.   ]
 [ 24.23    0.044  -0.   ]
 [ 13.78    0.      0.   ]
 [ -4.883  -0.     -0.   ]
 [ -0.      0.      0.   ]
 [-10.79   -0.026   0.   ]
 [  0.      0.      0.   ]
 [ 24.842   0.     -0.   ]
 [  2.194   0.      0.   ]]


# Incremental Fit

In [5]:
online_gamlss_lasso = rolch.OnlineGamlss(
    distribution=rolch.DistributionT(),
    method="lasso",
    estimation_kwargs={"ic": {i: "bic" for i in range(dist.n_params)}},
)

online_gamlss_lasso.fit(y[:-1], X[:-1, :], X[:-1, :], X[:-1, :])

print("Coefficients for the first N-1 observations \n")
print(np.vstack(online_gamlss_lasso.betas).T)

Coefficients for the first N-1 observations 

[[152.023   3.917   2.737]
 [ -0.     -0.      0.   ]
 [-10.774  -0.      0.   ]
 [ 24.566   0.035  -0.277]
 [ 14.204   0.     -0.   ]
 [ -5.537   0.      0.   ]
 [ -0.     -0.      0.   ]
 [-10.837  -0.      0.   ]
 [  0.      0.     -0.   ]
 [ 25.509   0.     -0.   ]
 [  1.748   0.     -0.143]]


In [6]:
online_gamlss_lasso.update(y[[-1]], X[[-1], :], X[[-1], :], X[[-1], :])

print("\nCoefficients after update call \n")
print(np.vstack(online_gamlss_lasso.betas).T)


Coefficients after update call 

[[152.144   3.933   3.475]
 [ -0.71   -0.045   0.511]
 [-12.52   -0.098   0.618]
 [ 24.582   0.037  -0.223]
 [ 15.34    0.066  -0.232]
 [-33.312  -0.407   2.879]
 [ 19.224   0.384  -2.365]
 [  2.271  -0.004  -0.51 ]
 [  7.308  -0.089   0.267]
 [ 35.061   0.189  -1.479]
 [  2.397   0.044  -0.235]]
